In [12]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [13]:
run_path = "data_clean/runs/2024-10-20_18-55-44/PCA_var_>_0.95,_nan_<_0.4_results.pkl"
with open(run_path, "rb") as f:
    runs = pickle.load(f)
print(json.dumps(runs, indent=4, default=str))

{
    "Logistic Regression": {
        "model_cls": "<class 'models.LogisticRegression'>",
        "hyperparam_search": {
            "gamma": [
                null
            ],
            "use_line_search": [
                true
            ],
            "optim_algo": [
                "lbfgs"
            ],
            "optim_kwargs": [
                {
                    "epochs": 1
                }
            ],
            "class_weights": [
                {
                    "0": 1,
                    "1": 1
                },
                {
                    "0": 1,
                    "1": 2
                },
                {
                    "0": 1,
                    "1": 4
                }
            ],
            "reg_mul": [
                0
            ],
            "verbose": [
                false
            ]
        },
        "all_val_scores": [
            0.22049661431540746,
            0.37206904188026285,
            0.42484238957

## Analysis